In [3]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON
import pandas as pd

In [ ]:
DATASET_PATH = 'processed/mfcc/mfcc_16000.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname='mfcc')

In [ ]:
def create_dense_model():
    # build the network architecture
    model = keras.Sequential([
        # input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),

        # 1st hidden layer
        keras.layers.Dense(512, activation='relu'),

        # 2nd hidden layer
        keras.layers.Dense(256, activation='relu'),

        # 3rd hidden layer
        keras.layers.Dense(128, activation='relu'),

        # output layer
        keras.layers.Dense(len(mapping), activation='softmax')
    ])

    return model

In [ ]:
# SPLIT DOS DADOS
random_state = 42
# for random_state in [5438, 53, 14]:
#     for _ in range(4):
# split data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

In [ ]:
# SALVA UMA CÓPIA DOS DADOS DE TESTE

# data = {
#     "mapping": mapping,
#     "labels": targets_test,
#     "mfcc": inputs_test,
# }

# JSON.create_json_file(f'datatest/deep/datatest_{random_state}_{inputs.shape[0]}.json', data)

In [ ]:
# CRIA O MODELO
model = create_dense_model()

# COMPILA A REDE
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

In [ ]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{timestamp}')

JSON.create_json_file(f'models/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [ ]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
            validation_data=(inputs_test, targets_test),
            epochs=10000,
            batch_size=128,
            callbacks=[earlystopping_cb, mdlcheckpoint_cb])

In [ ]:
# GERA O GRAFICO DE ACURÁCIA
pd.DataFrame(history.history).plot(figsize=(10,6))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.title('Model training')
plt.savefig(f'models/{timestamp}/model_history.png')
plt.close()

# # GERA O GRÁFICO DE PERCA
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.savefig(f'models/{timestamp}/graph_loss.png')
# plt.close()

In [ ]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

In [ ]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{timestamp}',
                   f'models/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')